In [24]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [60]:
def load_and_preprocess_data(directory):
    audio_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            filepath = os.path.join(directory, filename)
            waveform, sample_rate = librosa.load(filepath, sr=None)
            audio_data.append(waveform)
    return audio_data

import os

def create_spectrogram(waveform, save_path):
    _, _, spec, _ = plt.specgram(waveform, Fs=2, cmap="viridis")

    if spec.ndim == 1:
        # If spec is 1D, reshape it to 2D
        spec = np.expand_dims(spec, axis=0)

    spec[spec < 0] = 1e-10  # Set negative values to a small positive constant
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    plt.imshow(10 * np.log10(spec), cmap="viridis")
    plt.axis("off")
    plt.savefig(save_path)
    plt.close()

def load_data(transcript_file_path, spectrogram_directory):
    if not os.path.exists(transcript_file_path):
        raise FileNotFoundError(f"Transcript file not found: {transcript_file_path}")
    
    spectrogram_data = []
    transcripts = []

    with open(transcript_file_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            # Split the line at the | character and take the first part as the filename
            filename = line.split("|")[0].strip()
            img_path = os.path.join(spectrogram_directory, filename)
            
            # Add print statements for debugging
            print(f"Checking file: {img_path}")

            if os.path.exists(img_path):
                transcripts.append(filename)
                img = load_img(img_path, color_mode="grayscale", target_size=(time_steps, num_mel_bins))
                spectrogram = img_to_array(img) / 255.0
                spectrogram_data.append(spectrogram)
            else:
                print(f"Spectrogram file not found: {img_path}")

    if not spectrogram_data or not transcripts:
        raise ValueError("No valid data found.")

    return np.array(spectrogram_data), np.array(transcripts)

# Load and preprocess audio data
train_data = load_and_preprocess_data("E:\\ai\\audiomodel\\wav\\train_dataset")
val_data = load_and_preprocess_data("E:\\ai\\audiomodel\\wav\\val_dataset")
test_data = load_and_preprocess_data("E:\\ai\\audiomodel\\wav\\test_dataset")
transcript_file_path = "E:\\ai\\audiomodel\\wav\\transcript.txt"
spectrogram_directory = "E:\\ai\\audiomodel\\wav\\spectrograms"

try:
    train_features, train_labels = load_data(transcript_file_path, spectrogram_directory)
except Exception as e:
    print(f"An error occurred: {e}")

Checking file: E:\ai\audiomodel\wav\spectrograms\segment_1
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_1
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_2
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_2
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_3
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_3
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_4
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_4
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_5
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_5
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_6
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_6
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_7
Spectrogram file not found: E:\ai\audiomodel\wav\spectrograms\segment_7
Checking file: E:\ai\audiomodel\wav\spectrograms\segment_8
Spectrogram file not fou

In [58]:


# Create and save spectrograms
for i, waveform in enumerate(train_data):
    save_path = os.path.join(r"E:\ai\audiomodel\wav\spectrograms\train_dataset", f"train_spec_{i}.png")
    create_spectrogram(waveform, save_path)

# Load spectrogram data
train_features, train_labels = load_data(r"E:\ai\audiomodel\wav\spectrograms\train_dataset")
val_features, val_labels = load_data(r"E:\ai\audiomodel\wav\spectrograms\val_dataset")

# Create the model
input_shape = (time_steps, num_mel_bins)
output_shape = num_audio_samples
tacotron = tacotron_model(input_shape, output_shape)

# Compile the model
tacotron.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
num_epochs = 10
tacotron.fit(train_features, train_labels, epochs=num_epochs, validation_data=(val_features, val_labels))

# Save the model
tacotron.save("tts_model.h5")


TypeError: load_data() missing 1 required positional argument: 'spectrogram_directory'